In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


### Import the QmioBackend code

In [2]:
from qmiotools.integrations.qiskitqmio import QmioBackend

In [ ]:
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
from qiskit.visualization import plot_gate_map

### Create an instance of the QMIO backend, loading he last calibration from the directory given by the QMIO_CALIBRATIONS environ

In [2]:
backend=QmioBackend()

Importing calibrations from  /home/cesga/agomez/QMIO/calibrations/2024_05_11__13_27_21.json


In [ ]:
%matplotlib inline
c=QuantumCircuit(2)
c.h(0)
c.cx(0,1)
c.measure_all()
c.draw(output="mpl")

In [ ]:
d=c.copy()
d.draw(output="mpl")

In [ ]:

qc_basis = transpile(c, backend)
qc_basis.draw(output="mpl")

In [ ]:
qd_basis = transpile(d, backend)
qd_basis.draw(output="mpl")

# These cells only work if executed in a node with connection to the QPU

In [ ]:
job=backend.run([c,d], shots=100000)
job.result().get_counts(c)
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Without Optimisation")

In [ ]:
job=backend.run([qc_basis,qd_basis])
print(job.result().get_counts(qc_basis))
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(),title="With Optimisation")

In [ ]:
print(backend.target.build_coupling_map())

In [ ]:
import numpy as np
qc = QuantumCircuit(2, 2)
 
qc.h(0)
qc.x(1)
qc.cp(np.pi/4, 0, 1)
qc.h(0)
qc.measure([0], [0])
qc.measure([1], [1])
 
qc_basis = transpile(qc, backend)
qc_basis.name=c.name+"_opt"
qc_basis.draw(output="mpl")

In [ ]:
job=backend.run([qc,qc_basis])
job.result().get_counts(qc)
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Comparing Optimisation")

In [ ]:
nqubits=3
ghz = QuantumCircuit(nqubits)
ghz.h(0)
ghz.cx(0, range(1, nqubits))
ghz.measure_all()
ghz_0=transpile(ghz,backend,optimization_level=0)
job=backend.run(ghz_0)
job.result().get_counts(ghz_0)
plot_histogram(job.result().get_counts(), title="Comparing Optimisation")

In [ ]:
ghz_1=transpile(ghz,backend,optimization_level=1)
ghz_1.name=ghz_1.name+"_1"
ghz_2=transpile(ghz,backend,optimization_level=2)
ghz_2.name=ghz_1.name+"_2"
ghz_3=transpile(ghz,backend,optimization_level=3)
ghz_3.name=ghz_1.name+"_3"
job=backend.run([ghz_0,ghz_1,ghz_2,ghz_3])
plot_histogram(job.result().get_counts(), title="Comparing Optimisation",legend=[ghz_0.name,ghz_1.name,ghz_2.name,ghz_3.name])

In [ ]:
pos=[(1,2),(1,3),(1,4),(1,5),(1,6),
            (2,6.5),(2,4.5),(2,2.5),
            (3,1),(3,2),(3,3),(3,4),(3,5),
            (4,5.5),(4,3.5),(4,1.5),
            (5,1),(5,2),(5,3),(5,4),(5,5),(5,6),
            (6,6.5),(6,4.5),(6,2.5),(6,1),
            (7,1),(7,2),(7,3),(7,4),(7,5),(7,6)]
plot_gate_map(backend, qubit_coordinates=pos)

In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap
 
#coupling_map = CouplingMap.from_heavy_hex(3)

 
ghz = QuantumCircuit(15)
ghz.h(0)
ghz.cx(0, range(1, 15))
 
depths = []
for i in range(20):
    depths.append(
        transpile(
            ghz,
            backend,
            seed_transpiler=i,
            layout_method='trivial'  # Fixed layout mapped in circuit order
        ).depth()
    )
 
plt.figure(figsize=(8, 6))
plt.hist(depths, align='left', color='#AC557C')
plt.xlabel('Depth', fontsize=14)
plt.ylabel('Counts', fontsize=14);